In [2]:
%matplotlib qt5
# %matplotlib notebook

import time
import os
from datetime import datetime

import numpy as np

import matplotlib.pyplot as mplt
import matplotlib.gridspec as mgs

import pyaccel as pa

from siriuspy.search import IDSearch
from siriuspy.devices import IVU, SOFB, HLFOFB, IDFF, DVFImgProc, \
    Tune, PowerSupply
from apsuite.commisslib.meas_ac_orm import MeasACORM
from apsuite.commisslib.meas_bpms_signals import AcqBPMsSignals

from mathphys.functions import save, load

mplt.rcParams.update({
    'font.size': 12,'lines.linewidth': 2,
    'axes.grid': True, 'grid.alpha': 0.5,
    'grid.linestyle': '--', 'text.usetex': True})

# Define functions

In [ ]:
def initialize_ivu(beamline):
    # Search ID
    devnameivu = IDSearch.conv_beamline_2_idname(beamline=beamline)
    ivu = IVU(devname=devnameivu)

    # Disable beamline control
    ivu.cmd_beamline_ctrl_disable()
    print('beamline control: ', ivu.is_beamline_ctrl_enabled)

    # Disable center mode and pitch mode
    ivu.set_center_mode(False)
    ivu.set_pitch_mode(False)
    time.sleep(0.5)
    print('center mode: ', ivu.center_mode_status)
    print('pitch mode: ', ivu.pitch_mode_status)

    # Set gap speed
    ivu.set_gap_speed(0.1)
    time.sleep(0.5)
    print('gap speed: {:.3f} mm/s'.format(ivu.gap_speed))

    # Set gap to parked condition
    ivu.set_gap(ivu.kparameter_parked)
    time.sleep(0.5)
    print('gap: {:.3f} mm'.format(ivu.gap))

    return ivu

def move_ivu_gap(ivu:IVU, gap, timeout, verbose=False):
    ivu.set_gap(gap)
    time.sleep(0.5)
    print('Gap-RB {:.3f} mm'.format(ivu.gap)) if verbose else 0
    if ivu.cmd_move_gap_start(timeout):
        time.sleep(0.5)
        print('Undulator is moving...') if verbose else 0
        while ivu.is_moving:
            time.sleep(0.1)
            print('Current gap {:.3f} mm.'.format(ivu.gap_mon), end='\r') if verbose else 0
        print('Gap {:.3f} mm reached.'.format(ivu.gap)) if verbose else 0
        return True
    else:
        print('Error while cmd_move_start.')
        return False

def move_ivu_gap_robust(ivu:IVU, gap, timeout, maxiter=3, verbose=False):
    sucess = move_ivu_gap(ivu, gap=gap, timeout=timeout, verbose=verbose)
    i=0
    while not sucess and i<maxiter:
        i += 1
        print('Trying {:0f}/{:.0f}'.format(i, maxiter)) if verbose else 0
        ivu.cmd_reset(timeout=timeout)
        time.sleep(0.5)
        sucess = move_ivu_gap(ivu, gap=gap, timeout=timeout, verbose=verbose)
    if sucess:
        print('Movimentation done!\n')
        return True
    else:
        print('Error while moving.\n')
        return False

def meas_acorm(meas_orm, sofb, nr_trials=4):
    for trial in range(nr_trials):
        sofb.correct_orbit_manually(nr_iters=5, residue=2)
        meas_orm.start()
        if not meas_orm.wait_measurement(timeout=4*60):
            print(f'  ACORM timeout (trial={trial:02d}).')
            meas_orm.stop()
            if not meas_orm.wait_measurement(4*60):
                raise Exception('Experiment is not finishing.')
        if not meas_orm.check_measurement_finished_ok():
            print(f'  ACORM not finished ok (trial={trial:02d}).')
            continue
        meas_orm.process_data()
        if meas_orm.check_measurement_quality():
            break
        else:
            print(f'  ACORM data not ok (trial={trial:02d}).')
    else:
        raise Exception('Maximum number of trials reached.')

# Create objects

In [ ]:
timeout = 3

meas_orm = MeasACORM(isonline=True)
sofb = SOFB(SOFB.DEVICES.SI)
fofb = HLFOFB(HLFOFB.DEVICES.SI)

# Make sure SOFB and FOFB are closed
sofb.cmd_turn_off_autocorr()
sofb.cmd_change_opmode_to_sloworb(timeout=timeout)
fofb.cmd_turn_off_loop_state()

beamline='EMA'
ivu = initialize_ivu(beamline)

In [ ]:
# Move IVU to its maximum gap
move_ivu_gap_robust(ivu, gap=24, timeout=timeout, verbose=True)

# Configure AC ORM meas

In [ ]:
meas_orm.params.timeout_bpms = 60
meas_orm.params.timeout_correctors = 20
meas_orm.params.ref_respmat_name = 'ref_respmat'
meas_orm.params.meas_bpms_noise = True
meas_orm.params.meas_rf_line = True

meas_orm.params.corrs_ch2meas = 'all'
meas_orm.params.corrs_cv2meas = 'all'
meas_orm.params.corrs_nrruns_per_acq = 4
meas_orm.params.corrs_excit_time = 4.000
meas_orm.params.corrs_delay = 5e-3
meas_orm.params.corrs_norm_kicks = True
meas_orm.params.corrs_ch_kick = 5.000
meas_orm.params.corrs_cv_kick = 5.000
meas_orm.params.corrs_dorb1ch = 40.000
meas_orm.params.corrs_dorb1cv = 40.000

nrsecs = 1
primes = meas_orm.params.find_primes(2*8*nrsecs + 2, 3)
primes = np.array(sorted(set(primes) - {59, 61}))
cv_freqs = primes[:8*nrsecs]
primes = np.array(sorted(set(primes) - set(cv_freqs)))
ch_freqs = primes[:6*nrsecs]
meas_orm.params.corrs_ch_freqs = ch_freqs
meas_orm.params.corrs_cv_freqs = cv_freqs

meas_orm.params.rf_excit_time = 1.000
meas_orm.params.rf_mode = 'Phase'
meas_orm.params.rf_step_kick = 5.000
meas_orm.params.rf_step_delay = 0.200
meas_orm.params.rf_phase_amp = 2.000

In [ ]:
print(meas_orm.params)

#### Test ACORM params

In [ ]:
meas_orm.start()

In [ ]:
meas_orm.ismeasuring

In [ ]:
meas_orm.stop()

In [ ]:
meas_orm.check_measurement_finished_ok()

In [ ]:
meas_orm.process_data()

In [ ]:
meas_orm.check_measurement_quality()

In [ ]:
mat_ac = meas_orm.build_respmat()

In [ ]:
mat_dc = meas_orm.get_ref_respmat()

In [ ]:
# Scale Factors
fig, ax = meas_orm.plot_scale_conversion_factors()

In [ ]:
# Correlation residues
fig, axs, corr = meas_orm.plot_comparison_correlations(mat_ac=mat_ac, mat_dc=mat_dc)
corr_h = corr[:, :120]
corr_v = corr[:, 120:280]
corr_rf = corr[:, -1]

idcsh = corr_h[1].argsort()
idcsv = corr_v[0].argsort() + 120

In [ ]:
fig, axs = meas_orm.plot_comparison_single_corrector(idcsh[-1], mat_ac=mat_ac, mat_dc=mat_dc)
fig, axs = meas_orm.plot_comparison_single_corrector(idcsv[-1], mat_ac=mat_ac, mat_dc=mat_dc)

In [ ]:
# Analyse Fitting
fig, axs, *orbs = meas_orm.plot_orbit_residue_after_fitting(bpm_idx=0, excit_idx=0, time_domain=True)

In [ ]:
fig, axs, *orbs = meas_orm.plot_orbit_residue_after_fitting(bpm_idx=0, excit_idx=0, time_domain=False)

# Measure ACORM for different gaps

In [6]:
gaps = np.arange(24, 4, -1)

In [ ]:
for i, gap in enumerate(gaps):

    # Move gap
    sucess = move_ivu_gap_robust(ivu, gap=gap, timeout=timeout, verbose=True)
    
    # Verify if undulator moved
    if not sucess:
        break
    
    kpar_stg = f'{gap:.3f}'.replace('.', 'p')
    name = f'IVU_{beamline:s}_'
    name += f'ACORM_kparam_{kpar_stg:s}'

    meas_acorm(meas_orm, sofb, nr_trials=4)
    meas_orm.save_loco_input_data(name)